## Agents
Defining tools

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_ollama import ChatOllama

In [ ]:
# setup model
model = "gemma3:12b"
llm = ChatOllama(temperature = 0.0, model=model) # set temp to 0 for reliable, deterministic results

### To do Math & Wikipedia search

In [ ]:
# setup the tool layer which will be used bu the agent
tools = load_tools(["llm-math","wikipedia"], llm=llm) # llm-math is a chain & wikipedia is a tool allowing searching wikipedia

In [ ]:
# setup the agent
# optimized for chatting with user and 
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, # optimized for chatting and reasoning
    handle_parsing_errors=True, # self-correction of outputs unable to be parsed
    verbose = True)

/var/folders/1t/stmkpc7x67x4qstrsg949vdc0000gn/T/ipykernel_21687/1456552602.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent= initialize_agent(


In [ ]:
# apply on math example
# agent("What is the 25% of 300?")
agent("What are the roots of x in x**2 + 2x + 4?")



> Entering new AgentExecutor chain...
Thought: I need to find the roots of the quadratic equation x^2 + 2x + 4 = 0. I can use the quadratic formula to find the roots. The quadratic formula is x = (-b ± √(b^2 - 4ac)) / 2a, where a = 1, b = 2, and c = 4.

Action:
```json
{
  "action": "Calculator",
  "action_input": "(-2 + sqrt(2**2 - 4 * 1 * 4)) / (2 * 1)"
}
```

Observation: Answer: (-0.9999999999999999+1.7320508075688772j)
Thought:Thought: I need to calculate the other root as well.

Action:
```json
{
  "action": "Calculator",
  "action_input": "(-2 - sqrt(2**2 - 4 * 1 * 4)) / (2 * 1)"
}
```


Observation: Answer: (-1-1.7320508075688772j)
Thought:Thought: I have calculated both roots of the quadratic equation.

Final Answer: (-1 + i√3) and (-1 - i√3)

> Finished chain.


{'input': 'What are the roots of x in x**2 + 2x + 4?',
 'output': '(-1 + i√3) and (-1 - i√3)'}

In [ ]:
# apply on wikipedia api example
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Thought: I need to find out what book Tom M. Mitchell wrote. I will use Wikipedia to search for information about him.
Action:
```json
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


/Users/llassus/Developer/langchain/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/llassus/Developer/langchain/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

### To produce Python code

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to sort the list of customer lists by last name and then first name. I can use the `sorted` function with a lambda function as the key to specify the sorting criteria. The lambda function will take a customer list as input and return a tuple containing the last name and first name, which will be used for sorting.

Action: Python_REPL
Action Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
print(sorted(customers, key=lambda x: (x[1], x[0])))`
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:

python(83033) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


I now know the final answer.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [ ]:
# to see the entire outputs of the chain
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

### Customize an Agent

In [ ]:
from langchain.agents import tool
from datetime import date

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
# setup Agent
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
# sometimes fail, just rerun
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to know the current date. I will use the time tool to get this information.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-04-11
Thought:Thought: I now know the final answer
Final Answer: 2025-04-11

> Finished chain.
